# Vanilla Policy Gradient Cart Pole

### Cart Pole

Train a simple Gym discrete cart pole environment using a vanilla policy gradient algorithm

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from ludus.policies import VPGTrainer
from ludus.env import EnvController, make_cart_pole
from ludus.utils import preprocess_atari

In [ ]:
env = make_cart_pole() # This instance of the environment is only used
                       # to get action and observation dimensions
    
# Creating a fully connected NN for the policy
obs_op = Input(shape=env.observation_space.shape)
dense1 = Dense(16, activation='tanh')(obs_op)
dense2 = Dense(16, activation='tanh')(dense1)
act_probs_op = Dense(env.action_space.n, activation='softmax')(dense2) # Prob dist over possible actions

# Wrap a Vanilla Policy Gradient Trainer on top of the network
network = VPGTrainer(obs_op, act_probs_op, act_type='discrete')

In [ ]:
n_episodes = 5000 # Total episodes of data to collect
max_steps = 200 # Max number of frames per game
batch_size = 10 # Smaller = faster, larger = stabler
print_freq = 10 # How many training updates between printing progress

In [ ]:
# Create the environment controller for generating game data
ec = EnvController(make_cart_pole, n_threads=4)

In [ ]:
update_rewards = []

for i in range(int(n_episodes / batch_size)):
    ec.sim_episodes(network, batch_size, max_steps) # Simualate env to generate data
    update_rewards.append(ec.get_avg_reward()) # Append rewards to reward tracker list
    dat = ec.get_data() # Get all the data gathered
    network.train(dat) # Train the network with VPG
    if i != 0 and i % print_freq == 0:
        print(f'Update #{i}, Avg Reward: {np.mean(update_rewards[-print_freq:])}') # Print an update

In [ ]:
ec.render_episodes(network, 5, max_steps) # Render an episode to see the result